In [62]:
# Import required libraries

import pandas as pd
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline


In [63]:
# Load the environment variables from the .env file

load_dotenv()


True

In [64]:
#Create list of tickers for the bond portfolio

tickers = ["EEM", "VOO", "VTI", "VTV", "VUG"]


#Review bond portfolio tickers

tickers


['EEM', 'VOO', 'VTI', 'VTV', 'VUG']

In [65]:
# Set the variables for the Alpaca API and secret keys

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


# Create the Alpaca tradeapi.REST object

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
    

In [66]:
# Set timeframe to 1D 

timeframe = "1D"


# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day

start_date = pd.Timestamp("1991-10-22", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-11-03", tz="America/New_York").isoformat()


In [67]:
# Use the Alpaca get_barset function to get current closing prices the bond portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame

bonds_prices = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
).df


# Review the first 5 rows of the bond tickers Alpaca DataFrame

bonds_prices.head()


EEM                                          VOO  \
                            open     high     low   close    volume      open   
time                                                                            
2021-06-15 00:00:00-04:00  55.30  55.3100  54.940  55.060  26957077  391.1500   
2021-06-16 00:00:00-04:00  54.96  55.0950  54.090  54.290  40167671  390.4186   
2021-06-17 00:00:00-04:00  54.54  54.8199  54.450  54.625  25071984  387.6400   
2021-06-18 00:00:00-04:00  54.38  54.4800  54.125  54.200  34290029  384.7000   
2021-06-21 00:00:00-04:00  54.17  54.4600  53.920  54.455  20824410  384.4400   

                                                              ...     VTV  \
                             high       low   close   volume  ...    open   
time                                                          ...           
2021-06-15 00:00:00-04:00  391.20  389.4099  390.27  2313601  ...  139.69   
2021-06-16 00:00:00-04:00  390.58  386.0200  388.14  3997776  ...  139.80   
2021-06-17 00:00:00-04:00  388.91  385.5000  387.94  3859781  ...  139.00   
2021-06-18 00:00:00-04:00  385.38  382.5200  382.80  4646596  ...  135.69   
2021-06-21 00:00:00-04:00  388.38  383.6500  388.19  3937896  ...  135.43   

                                                                  VUG  \
                               high      low   close   volume    open   
time                                                                    
2021-06-15 00:00:00-04:00  140.1400  139.290  139.85  2523622  278.23   
2021-06-16 00:00:00-04:00  139.8000  138.337  138.84  2341872  276.83   
2021-06-17 00:00:00-04:00  139.1100  136.350  137.13  2648960  274.89   
2021-06-18 00:00:00-04:00  135.8200  134.440  134.47  4036564  277.79   
2021-06-21 00:00:00-04:00  137.1164  135.370  137.06  2033090  277.50   

                                                                
                               high      low     close  volume  
time                                                            
2021-06-15 00:00:00-04:00  278.2300  276.205  276.7000  889339  
2021-06-16 00:00:00-04:00  277.7300  273.030  275.4014  796527  
2021-06-17 00:00:00-04:00  279.6498  274.835  279.0100  830911  
2021-06-18 00:00:00-04:00  278.6100  276.540  276.9400  850301  
2021-06-21 00:00:00-04:00  279.6200  275.510  279.4500  709649  

[5 rows x 25 columns]

In [68]:

# Create an empty DataFrame to populate the closing prices on the bonds

bonds_closing_prices = pd.DataFrame()


# Use for-loop to pull the tickers' closing prices and populate the bonds_closing_prices DataFrame

for ticker in tickers:
    bonds_closing_prices[ticker] = bonds_prices[ticker]["close"] 


# Review DataFrame

bonds_closing_prices


,EEM,VOO,VTI,VTV,VUG
time,,,,,
2021-06-15 00:00:00-04:00,55.060,390.2700,220.74,139.850,276.7000
2021-06-16 00:00:00-04:00,54.290,388.1400,219.65,138.840,275.4014
2021-06-17 00:00:00-04:00,54.625,387.9400,219.48,137.130,279.0100
2021-06-18 00:00:00-04:00,54.200,382.8000,216.60,134.470,276.9400
2021-06-21 00:00:00-04:00,54.455,388.1900,219.67,137.060,279.4500
...,...,...,...,...,...
2021-10-28 00:00:00-04:00,51.660,421.3150,236.53,142.930,312.6800
2021-10-29 00:00:00-04:00,50.920,422.1400,236.91,142.740,314.1900
2021-11-01 00:00:00-04:00,51.330,423.0100,238.10,143.310,314.3700


In [69]:
# Configure the Monte Carlo simulation to forecast 10 years cumulative returns
# The weights should be split evenly across bonds.
# Run 500 samples.

#MC_bonds = MCSimulation(
    #portfolio_data = bonds_closing_prices,
    #weights = [.2, .2, .2, .2, .2],
    #num_simulation = 500,
    #num_trading_days = 252*10
#)


# Review the simulation input data

#MC_bonds.portfolio_data.head()#